In [ ]:
import datetime
import pandas as pd
import math
import sys

In [ ]:
file_name = '2022_01_13_Spot1.xlsx'
option_file_name  = '2022_01_13_Options.xlsx'
rate = 0.2
 


In [ ]:
def trading_cost(type,A):
    Quantity= 25
    Brokerage = 20/Quantity
    Exchange_comission = 0.00053*A
    SEBI_Charges = 0.000001*A
    GST = 0.18*(Brokerage+Exchange_comission+SEBI_Charges)
    StampDuty=0
    STT = 0
    if(type=="Buy"):
       StampDuty = 0.00003*A
    if(type=="Sell"):
       STT = 0.0005*A
       
    cost = Brokerage + Exchange_comission + StampDuty + SEBI_Charges + GST + STT

    return round(cost,2)
     


In [ ]:
def final_stop(file):
    a = file.split('_')
    final = "20"+a[0]+"-"+a[1]+"-"+a[2]+" 15:25:00"
    return final

def CMPFromExcel(i,df):

    return float(df.iloc[i+1,10])

def TimeFromExcel(i,df):

    return df.iloc[i+1,0]

def CallPremFromExcel(strikePrice,time,df):

    for j in range(len(df)):
        if(df.iloc[j,6]==str(strikePrice) and df.iloc[j,0]==time):
            return float(df.iloc[j,10])

def getFirstCellC(strikePrice,time,df):

    for j in range(len(df)):
        if(df.iloc[j,6]==str(strikePrice) and df.iloc[j,0]==time):
            return j
    
def CallPremFromExcel2(i,df):


    return float(df.iloc[i,10])

def CallPremFromExcel2High(i,df):

    return float(df.iloc[i,8])
            
def PutPremFromExcel(strikePrice,time,df):

    for j in range(len(df)):
        if(df.iloc[j,6]==str(strikePrice) and  df.iloc[j,0]==time):
            return float(df.iloc[j,10])

def getFirstCellP(strikePrice,time,df):

    for j in range(len(df)):
        if(df.iloc[j,6]==str(strikePrice) and df.iloc[j,0]==time):
            return j
def PutPremFromExcel2High(i,df):

    return float(df.iloc[i,8])
            
def PutPremFromExcel2(i,df):
    
    return float(df.iloc[i,10])

def findJ(startTime,df):
    final_stopTime = final_stop(file_name)
    if(str(startTime).split()==str(final_stopTime).split()):
        sys.exit()
    for j in range(df.iloc[-1,13]):
    
        if(df.iloc[j,0]==startTime  ):
            return j+1

def today(date):
    date =str(date).split('-')
    month = int(date[1])
    day = int(date[2])
    year = int(date[0])
    a = datetime.date(year,month,day).weekday()
    if a == 0:
        x  = "Monday"

    if a == 1:
        x  ="Tuesday" 

    if a==2:
        x  ="Wednesday" 

    if a==3:
       x ="Thursday" 

    if a==4:
       x ="Friday"

    if a==5:
       x = "Saturday" 

    if a==6: 
        x  = "Sunday"
    return x
    


In [ ]:

final_stopTime = final_stop(file_name)

dfs = pd.read_excel(file_name,engine='openpyxl',dtype=object,header=None)

lotsize = 25
no_of_lot = 1
qty = no_of_lot*lotsize


n=1
j = 100
t = TimeFromExcel(j,dfs)
day = str(t).split()

stopTime = day[0]+" 15:25:00"
startTime = day[0]+" 09:20:00"

dfs = pd.read_excel(file_name,engine='openpyxl',dtype=object,header=None)
print("dataframes ready")

j = findJ(startTime,dfs)

cmp = CMPFromExcel(j,dfs)

t = TimeFromExcel(j,dfs)
print(t)
bufferZone = "|"+"".center(16)+"|"+"".center(22)+"|"+"".center(10)+"|"+"".center(12)+"|"+"".center(23)+"|"+"".center(25)+"|"+"".center(24)+"|"+"".center(20)+"|"+"".center(16)+"|"+"".center(15)+"|"+"".center(16)+"|"+"".center(21)+"|\n"
dfs = pd.read_excel(file_name,engine='openpyxl',dtype=object,header=None)
dfc = pd.read_excel(option_file_name,sheet_name='Call',engine='openpyxl',dtype=object,header=None)
dfp = pd.read_excel(option_file_name,sheet_name='Put',engine='openpyxl',dtype=object,header=None)




In [ ]:
# Formatting
header = "FINAL TABLE".center(200,'*')+"\n"+"-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------"+"\n"+"|    Contract    |         Time         |    Day   |  Quantity  |      Entry Price      |      Current Price      |       Stop Loss        |     Exit Price     |   Entry Cost   |   Exit Cost   |   Total Cost   |          P&L        |"+"    Entry Time   "+"|"+"\n"

all_Days_Table = ""

def start(j,prev_table,total,count,dfc,dfp,dfs,all_Days_Table):
    
    entrytime =(all_Days_Table.split("\n")[0]).split()[1]

    count+=1

    initial = j

    cmp = CMPFromExcel(j,dfs)
     
    t = TimeFromExcel(j,dfs)
    day = str(t).split()
    myWeekday = today(day[0])

    stopTime = day[0]+" 15:25:00"
    startTime = day[0]+ " " +entrytime
     


    zintermediate = (math.floor(cmp/100))*100
    x = cmp%100
    y = math.floor(x)

    if (y>=50):
        strikeprice = zintermediate+100
    else:
        strikeprice = zintermediate
    SP = str(strikeprice)

    iP = getFirstCellP(SP,t,dfp)
    iC = getFirstCellC(SP,t,dfc)

    PE = "CNXBAN "+str(strikeprice)+" PE"
    CE = "CNXBAN "+str(strikeprice)+" CE"



    execPremCall = CallPremFromExcel(SP,t,dfc)
    execPremCall = float(execPremCall)
    execPremPut = PutPremFromExcel(SP,t,dfp)
    execPremPut = float(execPremPut)

    Entry_CostCall  = trading_cost("Sell",execPremCall)
    Entry_CostPut = trading_cost("Sell",execPremPut)





    # Unrealised Profit / (Loss) = Executed Premium - Current Premium
    # Realised Profit / (Loss) = Executed Premium - Stop Loss Premium

    #                                     Premium realtime

    #                                       stop loss

    
    
    stlp_1C = execPremCall*(1+rate)
    stlp_1P = execPremPut*(1+rate)
    
    #stop loss trial
    putStoplossIsHit = False
    CallStoplossIsHit = False

    exitPremPut = 0
    exitPremCall = 0

    # str(qty).center(12)+"|"+


    
    
    while(True):

        
        if(PutPremFromExcel2High(iP,dfp)>=stlp_1P ):
            putStoplossIsHit = True
            
            exitTimePut = TimeFromExcel(j,dfs)
            exitPremPut = min(PutPremFromExcel2High(iP,dfp),stlp_1P+5)
            
            exitCostPut = trading_cost("Buy",exitPremPut)
            totalCostPut = Entry_CostPut +exitCostPut
            Real_Put_P_or_L = execPremPut - exitPremPut - totalCostPut


            break

        if(CallPremFromExcel2High(iC,dfc)>=stlp_1C):
       
            exitPremCall = min(CallPremFromExcel2High(iC,dfc),stlp_1C+5)
            exitTimeCall = TimeFromExcel(j,dfs)

            exitCostCall = trading_cost("Buy",exitPremCall)
            totalCostCall = exitCostCall+Entry_CostCall
            Real_Call_P_or_L = execPremCall - exitPremCall - totalCostCall

            break
        if(TimeFromExcel(j,dfs) == stopTime):

            putStoplossIsHit = True
            exitPremCall = CallPremFromExcel2(iC,dfc)
            exitPremPut = PutPremFromExcel2(iP,dfp)
            exitTimeCall =exitTimePut= TimeFromExcel(j,dfs)
            

            exitCostCall = trading_cost("Buy",exitPremCall)
            exitCostPut = trading_cost("Buy",exitPremPut)

            totalCostCall = Entry_CostCall+exitCostCall
            totalCostPut = Entry_CostPut +exitCostPut

            Real_Call_P_or_L = execPremCall - exitPremCall - totalCostCall
            Real_Put_P_or_L = execPremPut - exitPremPut - totalCostPut
            all_Days_Table = all_Days_Table +header +prev_table + "\n"+           "|"+PE.center(16)+"|"+TimeFromExcel(j,dfs).center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremPut).center(23)+"|"+"--".center(25)+"|"+str(stlp_1P).center(24)+"|"+str(PutPremFromExcel2(iP,dfp)).center(20)+"|"+str(Entry_CostPut).center(16)+"|"+str(exitCostPut).center(15)+"|"+str(round(totalCostPut,2)).center(16)+"|"+str(Real_Put_P_or_L*qty).center(21)+"|"+str(entrytime).center(17)+"|"+ "\n"+           "|"+CE.center(16)+"|"+TimeFromExcel(j,dfs).center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremCall).center(23)+"|"+"--".center(25)+"|"+str(stlp_1C).center(24)+"|"+str(exitPremCall).center(20)+"|"+str(Entry_CostCall).center(16)+"|"+str(exitCostCall).center(15)+"|"+str(round(totalCostCall,2)).center(16)+"|"+str(Real_Call_P_or_L*qty).center(21)+"|"+ "\n"+bufferZone+ "\n"+"|"+ "Total ".center(16)+"|"+"".center(22)+"|"+"".center(10)+"|"+"".center(12)+"|"+"".center(23)+"|"+"".center(25)+"|"+"".center(24)+"|"+"".center(20)+"|"+"".center(16)+"|"+"".center(15)+"|"+"".center(16)+"|"+str(((Real_Call_P_or_L+Real_Put_P_or_L)*qty+total)).center(21)+"|"+ "\n"+"-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------"+ "\n"+"Day Finished"+ "\n"

            if(count<5 and j+37<dfs.iloc[-1,13] ):
                if(stopTime == final_stopTime):
                    print(all_Days_Table)
                    print("ALL DAYS FINISHED")
                    sys.exit()
                t = TimeFromExcel(j+37,dfs)
                day = str(t).split()

                startTime = day[0]+ " " +entrytime
                 

                j = findJ(startTime,dfs)
                start(j,"",0,count,dfc,dfp,dfs,all_Days_Table)

            else:
                print(all_Days_Table)
                print("ALL DAYS FINISHED")
                sys.exit()




        Real_Call_P_or_L = execPremCall - CallPremFromExcel2(iC,dfc)
        Real_Put_P_or_L = execPremPut - PutPremFromExcel2(iP,dfp)


        iP =iP+n
        iC = iC+n
        j=j+n       # time.sleep(trialTime*60)


    if(putStoplossIsHit):





        while(True):
            


            
            if(CMPFromExcel(j,dfs)>=cmp):    # or prem1*(1.1)>=prem
                cmp = CMPFromExcel(j,dfs)
            
                Real_Call_P_or_L = execPremCall - CallPremFromExcel2(iC,dfc)


                
                if(CallPremFromExcel2High(iC,dfc)>=stlp_1C):

                    exitPremCall = min(CallPremFromExcel2High(iC,dfc),stlp_1C+5)
                    exitTimeCall = TimeFromExcel(j,dfs)

                    exitCostCall = trading_cost("Buy",exitPremCall)
                    totalCostCall = exitCostCall+Entry_CostCall
                    
                    Real_Call_P_or_L = execPremCall - exitPremCall - totalCostCall


                    break
                if(TimeFromExcel(j,dfs) == stopTime):

                        exitPremCall = CallPremFromExcel2(iC,dfc)
                        exitTimeCall = TimeFromExcel(j,dfs)

                        exitCostCall = trading_cost("Buy",exitPremCall)
                        totalCostCall = exitCostCall+Entry_CostCall

                        Real_Call_P_or_L = execPremCall - exitPremCall - totalCostCall
                        Real_Put_P_or_L = execPremPut - exitPremPut - totalCostPut
                        
                        all_Days_Table = all_Days_Table + header +prev_table+ "\n"+"|"+PE.center(16)+"|"+exitTimePut.center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremPut).center(23)+"|"+"--".center(25)+"|"+str(stlp_1P).center(24)+"|"+str(exitPremPut).center(20)+"|"+str(Entry_CostPut).center(16)+"|"+str(exitCostPut).center(15)+"|"+str(round(totalCostPut,2)).center(16)+"|"+str(Real_Put_P_or_L*qty).center(21)+"|"+str(entrytime).center(17)+"|"+ "\n"+"|"+CE.center(16)+"|"+exitTimeCall.center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremCall).center(23)+"|"+"--".center(25)+"|"+str(stlp_1C).center(24)+"|"+str(exitPremCall).center(20)+"|"+str(Entry_CostCall).center(16)+"|"+str(exitCostCall).center(15)+"|"+str(round(totalCostCall,2)).center(16)+"|"+str(Real_Call_P_or_L*qty).center(21)+"|"+ "\n"+bufferZone+ "\n"+"|"+ "Total ".center(16)+"|"+"".center(22)+"|"+"".center(10)+"|"+"".center(12)+"|"+"".center(23)+"|"+"".center(25)+"|"+"".center(24)+"|"+"".center(20)+"|"+"".center(16)+"|"+"".center(15)+"|"+"".center(16)+"|"+str(((Real_Call_P_or_L+Real_Put_P_or_L)*qty+total)).center(21)+"|"+ "\n"+"-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------"+ "\n"+"Day Finished"+ "\n"
                        
                        if(count<5 and j+37<dfs.iloc[-1,13]):
                            if(stopTime == final_stopTime):
                                print(all_Days_Table)
                                print("ALL DAYS FINISHED")
                                sys.exit()
                            t = TimeFromExcel(j+37,dfs)
                            day = str(t).split()

                            startTime = day[0]+ " " +entrytime
                             

                            j = findJ(startTime,dfs)


                            start(j,"",0,count,dfc,dfp,dfs,all_Days_Table)
                        else:
                            print(all_Days_Table)
                            print("ALL DAYS FINISHED")
                            sys.exit()


                    
            



            else:
                if(stlp_1C>CallPremFromExcel2(iC,dfc)*(rate+1)):
                    stlp_1C=CallPremFromExcel2(iC,dfc)*(rate+1)
                if(TimeFromExcel(j,dfs) == stopTime):

                    exitPremCall = CallPremFromExcel2(iC,dfc)
                    exitTimeCall = TimeFromExcel(j,dfs)

                    exitCostCall = trading_cost("Buy",exitPremCall)
                    totalCostCall = exitCostCall+Entry_CostCall

                    Real_Call_P_or_L = execPremCall - exitPremCall - totalCostCall
                    Real_Put_P_or_L = execPremPut - exitPremPut - totalCostPut

                    all_Days_Table = all_Days_Table + header + prev_table+ "\n"+"|"+PE.center(16)+"|"+exitTimePut.center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremPut).center(23)+"|"+"--".center(25)+"|"+str(stlp_1P).center(24)+"|"+str(exitPremPut).center(20)+"|"+str(Entry_CostPut).center(16)+"|"+str(exitCostPut).center(15)+"|"+str(round(totalCostPut,2)).center(16)+"|"+str(Real_Put_P_or_L*qty).center(21)+"|"+str(entrytime).center(17)+"|"+ "\n"+"|"+CE.center(16)+"|"+exitTimeCall.center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremCall).center(23)+"|"+"--".center(25)+"|"+str(stlp_1C).center(24)+"|"+str(exitPremCall).center(20)+"|"+str(Entry_CostCall).center(16)+"|"+str(exitCostCall).center(15)+"|"+str(round(totalCostCall,2)).center(16)+"|"+str(Real_Call_P_or_L*qty).center(21)+"|"+ "\n"+bufferZone+ "\n"+"|"+ "Total ".center(16)+"|"+"".center(22)+"|"+"".center(10)+"|"+"".center(12)+"|"+"".center(23)+"|"+"".center(25)+"|"+"".center(24)+"|"+"".center(20)+"|"+"".center(16)+"|"+"".center(15)+"|"+"".center(16)+"|"+str(((Real_Call_P_or_L+Real_Put_P_or_L)*qty+total)).center(21)+"|"+ "\n"+"-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------"+ "\n"+"Day Finished"+ "\n"
                        
                    if(count<5 and j+37<dfs.iloc[-1,13]):
                        if(stopTime == final_stopTime):
                            print(all_Days_Table)
                            print("ALL DAYS FINISHED")
                            sys.exit()
                        t = TimeFromExcel(j+37,dfs)
                        day = str(t).split()

                        stopTime = day[0]+" 15:25:00"
                        startTime = day[0]+ " " +entrytime
                         
                        j = findJ(startTime,dfs)

                        
                        start(j,"",0,count,dfc,dfp,dfs,all_Days_Table)
                    else:
                        print(all_Days_Table)
                        print("ALL DAYS FINISHED")
                        sys.exit()

                cmp = CMPFromExcel(j,dfs)
                Real_Call_P_or_L = execPremCall - CallPremFromExcel2(iC,dfc)


            j=j+n
            iP =iP+n
            iC = iC+n
            # time.sleep(trialTime*60)
            
    else:
   
        while(True):
            



            
            if(CMPFromExcel(j,dfs)<=cmp ): 
                cmp = CMPFromExcel(j,dfs)



                Real_Put_P_or_L = execPremPut - PutPremFromExcel2(iP,dfp)




                if(PutPremFromExcel2High(iP,dfp)>=stlp_1P):
                    putStoplossIsHit = True
                    exitPremPut =min(PutPremFromExcel2High(iP,dfp),stlp_1P+5)
                    exitTimePut = TimeFromExcel(j,dfs)

                    exitCostPut = trading_cost("Buy",exitPremPut)
                    totalCostPut = Entry_CostPut+exitCostPut
                    Real_Put_P_or_L = execPremPut - exitPremPut - totalCostPut

                    
                    break

                if(TimeFromExcel(j,dfs) == stopTime):
                        exitPremPut = PutPremFromExcel2(iP,dfp)
                        
                        exitTimePut = TimeFromExcel(j,dfs)
                        

                        exitCostPut = trading_cost("Buy",exitPremPut)
                        totalCostPut = Entry_CostPut+exitCostPut

                        Real_Call_P_or_L = execPremCall - exitPremCall - totalCostCall
                        Real_Put_P_or_L = execPremPut - exitPremPut - totalCostPut

                        all_Days_Table = all_Days_Table + header + prev_table + "\n"+"|"+PE.center(16)+"|"+exitTimePut.center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremPut).center(23)+"|"+"--".center(25)+"|"+str(stlp_1P).center(24)+"|"+str(exitPremPut).center(20)+"|"+str(Entry_CostPut).center(16)+"|"+str(exitCostPut).center(15)+"|"+str(round(totalCostPut,2)).center(16)+"|"+str(Real_Put_P_or_L*qty).center(21)+"|"+str(entrytime).center(17)+"|"+ "\n"+"|"+CE.center(16)+"|"+exitTimeCall.center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremCall).center(23)+"|"+"--".center(25)+"|"+str(stlp_1C).center(24)+"|"+str(exitPremCall).center(20)+"|"+str(Entry_CostCall).center(16)+"|"+str(exitCostCall).center(15)+"|"+str(round(totalCostCall,2)).center(16)+"|"+str(Real_Call_P_or_L*qty).center(21)+"|"+ "\n"+bufferZone+ "\n"+"|"+ "Total ".center(16)+"|"+"".center(22)+"|"+"".center(10)+"|"+"".center(12)+"|"+"".center(23)+"|"+"".center(25)+"|"+"".center(24)+"|"+"".center(20)+"|"+"".center(16)+"|"+"".center(15)+"|"+"".center(16)+"|"+str(((Real_Call_P_or_L+Real_Put_P_or_L)*qty+total)).center(21)+"|"+ "\n"+"-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------"+ "\n"+"Day Finished"+ "\n"
                        
                        if(count<5 and j+37<dfs.iloc[-1,13]):
                            if(stopTime == final_stopTime):
                                print(all_Days_Table)
                                print("ALL DAYS FINISHED")
                                sys.exit()
                            t = TimeFromExcel(j+37,dfs)
                            day = str(t).split()

                            stopTime = day[0]+" 15:25:00"
                            startTime = day[0]+ " " +entrytime
                             

                            j = findJ(startTime,dfs)
                            
                            start(j,"",0,count,dfc,dfp,dfs,all_Days_Table)
                        else:
                            print(all_Days_Table)
                            print("ALL DAYS FINISHED")
                            sys.exit()
                            

                    
                    
            else:
                cmp = CMPFromExcel(j,dfs)
                if(PutPremFromExcel2(iP,dfp)*(rate+1)<stlp_1P):
                    stlp_1P=PutPremFromExcel2(iP,dfp)*(rate+1)
                if(TimeFromExcel(j,dfs) == stopTime):

                    exitPremPut = PutPremFromExcel2(iP,dfp)
                    exitTimePut = TimeFromExcel(j,dfs)

                    exitCostPut = trading_cost("Buy",exitPremPut)
                    totalCostPut = Entry_CostPut+exitCostPut

                    Real_Call_P_or_L = execPremCall - exitPremCall - totalCostCall
                    Real_Put_P_or_L = execPremPut - exitPremPut - totalCostPut
                    
                    all_Days_Table = all_Days_Table + header + prev_table+"\n"+"|"+PE.center(16)+"|"+TimeFromExcel(j,dfs).center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremPut).center(23)+"|"+"--".center(25)+"|"+str(stlp_1P).center(24)+"|"+str(exitPremPut).center(20)+"|"+str(Entry_CostPut).center(16)+"|"+str(exitCostPut).center(15)+"|"+str(round(totalCostPut,2)).center(16)+"|"+str(Real_Put_P_or_L*qty).center(21)+"|"+str(entrytime).center(17)+"|"+ "\n"+"|"+CE.center(16)+"|"+exitTimeCall.center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremCall).center(23)+"|"+"--".center(25)+"|"+str(stlp_1C).center(24)+"|"+str(exitPremCall).center(20)+"|"+str(Entry_CostCall).center(16)+"|"+str(exitCostCall).center(15)+"|"+str(round(totalCostCall,2)).center(16)+"|"+str(Real_Call_P_or_L*qty).center(21)+"|"+ "\n"+bufferZone+ "\n"+"|"+ "Total ".center(16)+"|"+"".center(22)+"|"+"".center(10)+"|"+"".center(12)+"|"+"".center(23)+"|"+"".center(25)+"|"+"".center(24)+"|"+"".center(20)+"|"+"".center(16)+"|"+"".center(15)+"|"+"".center(16)+"|"+str(((Real_Call_P_or_L+Real_Put_P_or_L)*qty+total)).center(21)+"|"+ "\n"+"-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------"+ "\n"+"Day Finished"+ "\n"
                        
                    if(count<5 and j+37<dfs.iloc[-1,13]):
                        if(stopTime == final_stopTime):
                            print(all_Days_Table)
                            print("ALL DAYS FINISHED")
                            sys.exit()
                        t = TimeFromExcel(j+37,dfs)
                        day = str(t).split()

                        stopTime = day[0]+" 15:25:00"
                        startTime = day[0]+ " " +entrytime
                         

                        j = findJ(startTime,dfs)
                        

                        start(j,"",0,count,dfc,dfp,dfs,all_Days_Table)                    
                    else:
                        print(all_Days_Table)
                        print("ALL DAYS FINISHED")
                        sys.exit()

                
    
                Real_Put_P_or_L = execPremPut - PutPremFromExcel2(iP,dfp)



            j=j+n
            iP =iP+n
            iC = iC+n

    session_total = (Real_Call_P_or_L +Real_Put_P_or_L)*qty

    x = "Total = "+str(total)


    prev_table = prev_table +"\n"+str(bufferZone)+"|"+PE.center(16)+"|"+str(exitTimePut).center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremPut).center(23)+"|"+"--".center(25)+"|"+str(stlp_1P).center(24)+"|"+str(exitPremPut).center(20)+"|"+str(Entry_CostPut).center(16)+"|"+str(exitCostPut).center(15)+"|"+str(round(totalCostPut,2)).center(16)+"|"+str(Real_Put_P_or_L*qty).center(21)+"|"+str(entrytime).center(17)+"|"+"\n"+"|"+CE.center(16)+"|"+str(exitTimeCall).center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremCall).center(23)+"|"+"--".center(25)+"|"+str(stlp_1C).center(24)+"|"+str(exitPremCall).center(20)+"|"+str(Entry_CostCall).center(16)+"|"+str(exitCostCall).center(15)+"|"+str(round(totalCostCall,2)).center(16)+"|"+str(Real_Call_P_or_L*qty).center(21)+"|\n"
    total  = total+session_total

    if(TimeFromExcel(j,dfs) != str(stopTime)):
        all_Days_Table=all_Days_Table+header+prev_table+"\n"
        
        # if(j==initial):
        #     start(j+1,prev_table,total,count,dfc,dfp,dfs,all_Days_Table)
        # else:
        #     start(j+1,prev_table,total,count,dfc,dfp,dfs,all_Days_Table)

        if(stopTime == final_stopTime):
            print(all_Days_Table)
            print("ALL DAYS FINISHED")
            sys.exit()
        t = TimeFromExcel(j,dfs)

        day = str(t).split()
        temp = day[0].split('-')
        check=None
        while(check==None and count<5):
            if(int(temp[2])>=9):
                
                temp[2] = str(int(temp[2])+1)
            else:

                temp[2] = "0"+str(int(temp[2])+1)

            day[0] = temp[0]+"-"+temp[1]+"-"+temp[2]


            stopTime = day[0]+" 15:25:00"
            startTime = day[0]+ " " +entrytime
             

            j = findJ(startTime,dfs)
            check = j

        if(count==5):
            print(all_Days_Table)
            print("ALL DAYS FINISHED")
            sys.exit()


        else:

            start(j+1,"",0,count,dfc,dfp,dfs,all_Days_Table)


    if(TimeFromExcel(j,dfs) == stopTime):
        
        all_Days_Table = all_Days_Table + header + prev_table+ "\n"+"|"+PE.center(16)+"|"+exitTimePut.center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremPut).center(23)+"|"+"--".center(25)+"|"+str(stlp_1P).center(24)+"|"+str(exitPremPut).center(20)+"|"+str(Entry_CostPut).center(16)+"|"+str(exitCostPut).center(15)+"|"+str(round(totalCostPut,2)).center(16)+"|"+str(Real_Put_P_or_L*qty).center(21)+"|"+str(entrytime).center(17)+"|"+ "\n"+"|"+CE.center(16)+"|"+exitTimeCall.center(22)+"|"+myWeekday.center(10)+"|"+str(qty).center(12)+"|"+str(execPremCall).center(23)+"|"+"--".center(25)+"|"+str(stlp_1C).center(24)+"|"+str(exitPremCall).center(20)+"|"+str(Entry_CostCall).center(16)+"|"+str(exitCostCall).center(15)+"|"+str(round(totalCostCall,2)).center(16)+"|"+str(Real_Call_P_or_L*qty).center(21)+"|"+ "\n"+bufferZone+ "\n"+"|"+ "Total ".center(16)+"|"+"".center(22)+"|"+"".center(10)+"|"+"".center(12)+"|"+"".center(23)+"|"+"".center(25)+"|"+"".center(24)+"|"+"".center(20)+"|"+"".center(16)+"|"+"".center(15)+"|"+"".center(16)+"|"+str(((Real_Call_P_or_L+Real_Put_P_or_L)*qty+total)).center(21)+"|"+ "\n"+"-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------"+ "\n"+"Day Finished"+ "\n"

        if(count<5 and j+37<dfs.iloc[-1,13] ):
            if(stopTime == final_stopTime):
                print(all_Days_Table)
                print("ALL DAYS FINISHED")
                sys.exit()
        
            t = TimeFromExcel(j+37,dfs)
            day = str(t).split()

            stopTime = day[0]+" 15:25:00"
            startTime = day[0]+ " " +entrytime
             


            j = findJ(startTime,dfs)


        
            start(j,"",0,count,dfc,dfp,dfs,all_Days_Table)
 
        else:
            print(all_Days_Table)
            print("ALL DAYS FINISHED")
            
            sys.exit()




In [ ]:
# start(j,"",0,0,dfc,dfp,dfs,"")


In [ ]:
from threading import Thread
import threading

class CustomThread(Thread):
    def __init__(self, group=None, target=None, name=None,args={},kwargs ={},Verbose=None ):
        Thread.__init__(self, group,target,name,args,kwargs)
        self._return = None

    def run(self):
        if self._target is not None:
            self._return = self._target(*self._args,**self._kwargs)


        

In [ ]:
# TimeFromExcel(0,dfs)
# x=findJ("2022-01-07 09:20:00",dfs)
# print("[",end="")

# for i in range(105,446,5):
#     print(f'"{dfs[0][i-1]}"',end=',')
# print("]")
# print("[",end="")

# for i in range(610,951,5):
#     print(f'"{dfs[0][i-1]}"',end=',')
# print("]")
# print("[",end="")

# for i in range(1115,1456,5):
#     print(f'"{dfs[0][i-1]}"',end=',')
# print("]")
# print("[",end="")

# for i in range(1618,1959,5):
#     print(f'"{dfs[0][i-1]}"',end=',')
# print("]")
# print("[",end="")
# for i in range(2122,2463,5):
#     print(f'"{dfs[0][i-1].split()[1]}"',end=',')
# print("]")


In [ ]:
# print(findJ("2022-01-10 09:20:00",dfs),findJ("2022-01-10 15:00:00",dfs))

In [ ]:
# print(findJ("2022-01-11 09:20:00",dfs),findJ("2022-01-11 15:00:00",dfs))

In [ ]:
# print(findJ("2022-01-12 09:20:00",dfs),findJ("2022-01-12 15:00:00",dfs))

In [ ]:
# print(findJ("2022-01-13 09:20:00",dfs),findJ("2022-01-13 15:30:00",dfs))


In [ ]:
findJ("2022-01-07 15:00:00",dfs)

In [16]:
from threading import Thread
import threading
import time


times = ["09:20:00","09:25:00","09:30:00","09:35:00","09:40:00","09:45:00","09:50:00","09:55:00","10:00:00","10:05:00","10:10:00","10:15:00","10:20:00","10:25:00","10:30:00","10:35:00","10:40:00","10:45:00","10:50:00","10:55:00","11:00:00","11:05:00","11:10:00","11:15:00","11:20:00","11:25:00","11:30:00","11:35:00","11:40:00","11:45:00","11:50:00","11:55:00","12:00:00","12:05:00","12:10:00","12:15:00","12:20:00","12:25:00","12:30:00","12:35:00","12:40:00","12:45:00","12:50:00","12:55:00","13:00:00","13:05:00","13:10:00","13:15:00","13:20:00","13:25:00","13:30:00","13:35:00","13:40:00","13:45:00","13:50:00","13:55:00","14:00:00","14:05:00","14:10:00","14:15:00","14:20:00","14:25:00","14:30:00","14:35:00","14:40:00","14:45:00","14:50:00","14:55:00","15:00:00"]



times1 = ["2022-01-07 09:20:00","2022-01-07 09:25:00","2022-01-07 09:30:00","2022-01-07 09:35:00","2022-01-07 09:40:00","2022-01-07 09:45:00","2022-01-07 09:50:00","2022-01-07 09:55:00","2022-01-07 10:00:00","2022-01-07 10:05:00","2022-01-07 10:10:00","2022-01-07 10:15:00","2022-01-07 10:20:00","2022-01-07 10:25:00","2022-01-07 10:30:00","2022-01-07 10:35:00","2022-01-07 10:40:00","2022-01-07 10:45:00","2022-01-07 10:50:00","2022-01-07 10:55:00","2022-01-07 11:00:00","2022-01-07 11:05:00","2022-01-07 11:10:00","2022-01-07 11:15:00","2022-01-07 11:20:00","2022-01-07 11:25:00","2022-01-07 11:30:00","2022-01-07 11:35:00","2022-01-07 11:40:00","2022-01-07 11:45:00","2022-01-07 11:50:00","2022-01-07 11:55:00","2022-01-07 12:00:00","2022-01-07 12:05:00","2022-01-07 12:10:00","2022-01-07 12:15:00","2022-01-07 12:20:00","2022-01-07 12:25:00","2022-01-07 12:30:00","2022-01-07 12:35:00","2022-01-07 12:40:00","2022-01-07 12:45:00","2022-01-07 12:50:00","2022-01-07 12:55:00","2022-01-07 13:00:00","2022-01-07 13:05:00","2022-01-07 13:10:00","2022-01-07 13:15:00","2022-01-07 13:20:00","2022-01-07 13:25:00","2022-01-07 13:30:00","2022-01-07 13:35:00","2022-01-07 13:40:00","2022-01-07 13:45:00","2022-01-07 13:50:00","2022-01-07 13:55:00","2022-01-07 14:00:00","2022-01-07 14:05:00","2022-01-07 14:10:00","2022-01-07 14:15:00","2022-01-07 14:20:00","2022-01-07 14:25:00","2022-01-07 14:30:00","2022-01-07 14:35:00","2022-01-07 14:40:00","2022-01-07 14:45:00","2022-01-07 14:50:00","2022-01-07 14:55:00","2022-01-07 15:00:00"]

times2 = ["2022-01-10 09:20:00","2022-01-10 09:25:00","2022-01-10 09:30:00","2022-01-10 09:35:00","2022-01-10 09:40:00","2022-01-10 09:45:00","2022-01-10 09:50:00","2022-01-10 09:55:00","2022-01-10 10:00:00","2022-01-10 10:05:00","2022-01-10 10:10:00","2022-01-10 10:15:00","2022-01-10 10:20:00","2022-01-10 10:25:00","2022-01-10 10:30:00","2022-01-10 10:35:00","2022-01-10 10:40:00","2022-01-10 10:45:00","2022-01-10 10:50:00","2022-01-10 10:55:00","2022-01-10 11:00:00","2022-01-10 11:05:00","2022-01-10 11:10:00","2022-01-10 11:15:00","2022-01-10 11:20:00","2022-01-10 11:25:00","2022-01-10 11:30:00","2022-01-10 11:35:00","2022-01-10 11:40:00","2022-01-10 11:45:00","2022-01-10 11:50:00","2022-01-10 11:55:00","2022-01-10 12:00:00","2022-01-10 12:05:00","2022-01-10 12:10:00","2022-01-10 12:15:00","2022-01-10 12:20:00","2022-01-10 12:25:00","2022-01-10 12:30:00","2022-01-10 12:35:00","2022-01-10 12:40:00","2022-01-10 12:45:00","2022-01-10 12:50:00","2022-01-10 12:55:00","2022-01-10 13:00:00","2022-01-10 13:05:00","2022-01-10 13:10:00","2022-01-10 13:15:00","2022-01-10 13:20:00","2022-01-10 13:25:00","2022-01-10 13:30:00","2022-01-10 13:35:00","2022-01-10 13:40:00","2022-01-10 13:45:00","2022-01-10 13:50:00","2022-01-10 13:55:00","2022-01-10 14:00:00","2022-01-10 14:05:00","2022-01-10 14:10:00","2022-01-10 14:15:00","2022-01-10 14:20:00","2022-01-10 14:25:00","2022-01-10 14:30:00","2022-01-10 14:35:00","2022-01-10 14:40:00","2022-01-10 14:45:00","2022-01-10 14:50:00","2022-01-10 14:55:00","2022-01-10 15:00:00"]


times3 = ["2022-01-11 09:20:00","2022-01-11 09:25:00","2022-01-11 09:30:00","2022-01-11 09:35:00","2022-01-11 09:40:00","2022-01-11 09:45:00","2022-01-11 09:50:00","2022-01-11 09:55:00","2022-01-11 10:00:00","2022-01-11 10:05:00","2022-01-11 10:10:00","2022-01-11 10:15:00","2022-01-11 10:20:00","2022-01-11 10:25:00","2022-01-11 10:30:00","2022-01-11 10:35:00","2022-01-11 10:40:00","2022-01-11 10:45:00","2022-01-11 10:50:00","2022-01-11 10:55:00","2022-01-11 11:00:00","2022-01-11 11:05:00","2022-01-11 11:10:00","2022-01-11 11:15:00","2022-01-11 11:20:00","2022-01-11 11:25:00","2022-01-11 11:30:00","2022-01-11 11:35:00","2022-01-11 11:40:00","2022-01-11 11:45:00","2022-01-11 11:50:00","2022-01-11 11:55:00","2022-01-11 12:00:00","2022-01-11 12:05:00","2022-01-11 12:10:00","2022-01-11 12:15:00","2022-01-11 12:20:00","2022-01-11 12:25:00","2022-01-11 12:30:00","2022-01-11 12:35:00","2022-01-11 12:40:00","2022-01-11 12:45:00","2022-01-11 12:50:00","2022-01-11 12:55:00","2022-01-11 13:00:00","2022-01-11 13:05:00","2022-01-11 13:10:00","2022-01-11 13:15:00","2022-01-11 13:20:00","2022-01-11 13:25:00","2022-01-11 13:30:00","2022-01-11 13:35:00","2022-01-11 13:40:00","2022-01-11 13:45:00","2022-01-11 13:50:00","2022-01-11 13:55:00","2022-01-11 14:00:00","2022-01-11 14:05:00","2022-01-11 14:10:00","2022-01-11 14:15:00","2022-01-11 14:20:00","2022-01-11 14:25:00","2022-01-11 14:30:00","2022-01-11 14:35:00","2022-01-11 14:40:00","2022-01-11 14:45:00","2022-01-11 14:50:00","2022-01-11 14:55:00","2022-01-11 15:00:00"]


times4 = ["2022-01-12 09:20:00","2022-01-12 09:25:00","2022-01-12 09:30:00","2022-01-12 09:35:00","2022-01-12 09:40:00","2022-01-12 09:45:00","2022-01-12 09:50:00","2022-01-12 09:55:00","2022-01-12 10:00:00","2022-01-12 10:05:00","2022-01-12 10:10:00","2022-01-12 10:15:00","2022-01-12 10:20:00","2022-01-12 10:25:00","2022-01-12 10:30:00","2022-01-12 10:35:00","2022-01-12 10:40:00","2022-01-12 10:45:00","2022-01-12 10:50:00","2022-01-12 10:55:00","2022-01-12 11:00:00","2022-01-12 11:05:00","2022-01-12 11:10:00","2022-01-12 11:15:00","2022-01-12 11:20:00","2022-01-12 11:25:00","2022-01-12 11:30:00","2022-01-12 11:35:00","2022-01-12 11:40:00","2022-01-12 11:45:00","2022-01-12 11:50:00","2022-01-12 11:55:00","2022-01-12 12:00:00","2022-01-12 12:05:00","2022-01-12 12:10:00","2022-01-12 12:15:00","2022-01-12 12:20:00","2022-01-12 12:25:00","2022-01-12 12:30:00","2022-01-12 12:35:00","2022-01-12 12:40:00","2022-01-12 12:45:00","2022-01-12 12:50:00","2022-01-12 12:55:00","2022-01-12 13:00:00","2022-01-12 13:05:00","2022-01-12 13:10:00","2022-01-12 13:15:00","2022-01-12 13:20:00","2022-01-12 13:25:00","2022-01-12 13:30:00","2022-01-12 13:35:00","2022-01-12 13:40:00","2022-01-12 13:45:00","2022-01-12 13:50:00","2022-01-12 13:55:00","2022-01-12 14:00:00","2022-01-12 14:05:00","2022-01-12 14:10:00","2022-01-12 14:15:00","2022-01-12 14:20:00","2022-01-12 14:25:00","2022-01-12 14:30:00","2022-01-12 14:35:00","2022-01-12 14:40:00","2022-01-12 14:45:00","2022-01-12 14:50:00","2022-01-12 14:55:00","2022-01-12 15:00:00"]


times5 = ["2022-01-13 09:20:00","2022-01-13 09:25:00","2022-01-13 09:30:00","2022-01-13 09:35:00","2022-01-13 09:40:00","2022-01-13 09:45:00","2022-01-13 09:50:00","2022-01-13 09:55:00","2022-01-13 10:00:00","2022-01-13 10:05:00","2022-01-13 10:10:00","2022-01-13 10:15:00","2022-01-13 10:20:00","2022-01-13 10:25:00","2022-01-13 10:30:00","2022-01-13 10:35:00","2022-01-13 10:40:00","2022-01-13 10:45:00","2022-01-13 10:50:00","2022-01-13 10:55:00","2022-01-13 11:00:00","2022-01-13 11:05:00","2022-01-13 11:10:00","2022-01-13 11:15:00","2022-01-13 11:20:00","2022-01-13 11:25:00","2022-01-13 11:30:00","2022-01-13 11:35:00","2022-01-13 11:40:00","2022-01-13 11:45:00","2022-01-13 11:50:00","2022-01-13 11:55:00","2022-01-13 12:00:00","2022-01-13 12:05:00","2022-01-13 12:10:00","2022-01-13 12:15:00","2022-01-13 12:20:00","2022-01-13 12:25:00","2022-01-13 12:30:00","2022-01-13 12:35:00","2022-01-13 12:40:00","2022-01-13 12:45:00","2022-01-13 12:50:00","2022-01-13 12:55:00","2022-01-13 13:00:00","2022-01-13 13:05:00","2022-01-13 13:10:00","2022-01-13 13:15:00","2022-01-13 13:20:00","2022-01-13 13:25:00","2022-01-13 13:30:00","2022-01-13 13:35:00","2022-01-13 13:40:00","2022-01-13 13:45:00","2022-01-13 13:50:00","2022-01-13 13:55:00","2022-01-13 14:00:00","2022-01-13 14:05:00","2022-01-13 14:10:00","2022-01-13 14:15:00","2022-01-13 14:20:00","2022-01-13 14:25:00","2022-01-13 14:30:00","2022-01-13 14:35:00","2022-01-13 14:40:00","2022-01-13 14:45:00","2022-01-13 14:50:00","2022-01-13 14:55:00","2022-01-13 15:00:00"]


t1 = time.perf_counter()

x=findJ("2022-01-07 09:20:00",dfs)

threads = []

for i in range(103,446):
    a = dfs[0][i].split()[1]
    if(a in times):
        j = findJ(dfs[0][i],dfs)
        t = Thread(target=start,args=[j,"",0,0,dfc,dfp,dfs,dfs[0][i]+"\n"])
        t.start()
        t.join()
        threads.append(t)
        print(len(threads))



t2 = time.perf_counter()

print(t2-t1)





2022-01-07 09:20:00
**********************************************************************************************FINAL TABLE***********************************************************************************************
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|    Contract    |         Time         |    Day   |  Quantity  |      Entry Price      |      Current Price      |       Stop Loss        |     Exit Price     |   Entry Cost   |   Exit Cost   |   Total Cost   |          P&L        |    Entry Time   |

|                |                      |          |            |                       |                         |                        |                    |                |               |                |                     |
|CNXBAN 37900 PE | 2022-01-07 10:40:00  |  Friday  |     25     |         458.55        |            --   